In [28]:
from cartoframes.auth import set_default_credentials
from cartoframes import read_carto, to_carto
import geopandas as gpd
import pandas as pd
import os
import logging 
from shapely import geometry
import requests
import re
from bs4 import BeautifulSoup
import glob
from zipfile import ZipFile
import shutil
import urllib
import json

In [2]:
# get top-level logger object
logger = logging.getLogger()
for handler in logger.handlers: logger.removeHandler(handler)
# manually set level 
logger.setLevel(logging.INFO)
# print to console
console = logging.StreamHandler()
logger.addHandler(console)
logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')

In [21]:
data_dir = 'data'

In [33]:
# insert the url used to download the data from the source website

ids = ["133029","133026","132991","133048","133030","133017","132934","132906","132930"]

url_template= 'https://www.climatewatchdata.org/api/v1/data/ndc_content?indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&indicator_ids[]={}&per_page=1000'
url = url_template.format(*tuple(ids)) + '&page={}'
print(url)


https://www.climatewatchdata.org/api/v1/data/ndc_content?indicator_ids[]=133029&indicator_ids[]=133026&indicator_ids[]=132991&indicator_ids[]=133048&indicator_ids[]=133030&indicator_ids[]=133017&indicator_ids[]=132934&indicator_ids[]=132906&indicator_ids[]=132930&per_page=1000&page={}


In [36]:
paginated_dfs = []
# download the cata from the source
for i in range(1):
    req = requests.get(url)
    text_data= req.text
    json_dict= json.loads(text_data)
    df = pd.DataFrame.from_dict(json_dict['data'])
    paginated_dfs.append(df)

df = pd.concat(paginated_dfs)


In [37]:
df_countries = df.groupby(['iso_code3', 'indicator_id']).agg({'country':'first', 'sector':'first', 'subsector':'first', 'indicator_name':'first','id':'count'}).reset_index()
print(df_countries.head())

  iso_code3                      indicator_id      country        sector  \
0       AFG          a_coastal_fisheries_auto  Afghanistan  Coastal Zone   
1       AFG         a_coastal_management_auto  Afghanistan  Coastal Zone   
2       AFG       a_coastal_zone_general_auto  Afghanistan  Coastal Zone   
3       AFG  a_fisheries_and_aquaculture_auto  Afghanistan   Agriculture   
4       AFG                  a_mangroves_auto  Afghanistan  Coastal Zone   

                   subsector             indicator_name  id  
0          Coastal Fisheries          Coastal Fisheries   1  
1         Coastal management         Coastal management   1  
2      Coastal Zone: General      Coastal Zone: General   1  
3  Fisheries and Aquaculture  Fisheries and Aquaculture   1  
4                  Mangroves                  Mangroves   1  
